In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from feature_bagging import FeatureBaggingWithHyperparamTuning
import statsmodels.api as sm

from sklearn.preprocessing import MinMaxScaler
DATOS_DIR = '~/buckets/b1/datasets/'

In [2]:
df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 

In [3]:
# Función para escalar y devolver una serie
def minmax_scale_group(group):
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Almacenar el escalador para este grupo
    return pd.Series(scaled_values, index=group.index)

# Función para desescalar y devolver una serie
def inverse_minmax_scale_group(group):
    scaler = scalers[group.name]
    inversed_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    return pd.Series(inversed_values, index=group.index)

# Definir la métrica personalizada
def multinacional_metric(y_true, y_pred):
    return abs(sum(y_true - y_pred)) / sum(y_true)

In [4]:
scalers = {}

df_final['tn'] = df_final.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
df_final['tn_2'] = df_final.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado

In [5]:
nan_count = df_final.isnull().sum()
print(nan_count)
data = df_final.loc['2018-01-01':'2020-12-01']
#data = data[data['product_id'] == 20441] 
categorical_columns = data.select_dtypes(include=['object']).columns
data[categorical_columns] = data[categorical_columns].fillna('Sin Datos')
numeric_columns = data.select_dtypes(include=['number']).columns
data[numeric_columns] = data[numeric_columns].fillna(0)
data = data.copy()

#TEST
#true =df_final.loc['2019-09-01':'2019-10-01']
#true = true[true['product_id'] == 20441]



customer_id                  0
product_id                   0
plan_precios_cuidados        0
cust_request_qty             0
cust_request_tn              0
                            ..
descripcion_seda delicada    0
descripcion_sport            0
descripcion_sport extremo    0
descripcion_trigo            0
descripcion_zanahoria        0
Length: 1267, dtype: int64


C:\Users\Admin\AppData\Local\Temp\ipykernel_1164\3133238076.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numeric_columns] = data[numeric_columns].fillna(0)


In [8]:
data.columns = data.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)


In [9]:
predictions = []
# Define parameter bounds for Bayesian optimization
param_bounds = {
    'num_leaves': (10, 20),
    'learning_rate': (0.01, 0.1),
    'n_estimators': (50, 100 ),
    'min_child_samples': (5, 10),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'max_depth': (5, 10)
}
# Custom optimization target function
def multinacional_metric(y_true, y_pred):
     return abs(sum(y_true - y_pred)) / sum(y_true)


# Filtrar los datos del producto
X = data.drop(columns=['tn_2'])
X = X.copy()
y = data['tn_2']
y= y.copy()
# Create the feature bagging model
feature_bagging_model = FeatureBaggingWithHyperparamTuning(
    X, y, n_models=10, 
    feature_fraction=0.5, 
    sample_fraction=0.8, 
    param_bounds=param_bounds, 
    random_state=30000841,
    optimization_target=multinacional_metric)

# Fit the model with a single seed
feature_bagging_model.fit()
single_seed_predictions = feature_bagging_model.predict(X)

# Define multiple seeds
#seeds = [10000019, 20000379, 30000841, 40001387, 50001863]
    # Fit the model with multiple seeds and get combined predictions
#combined_predictions = feature_bagging_model.fit_multiple_seeds(seeds)

# Realizar el pronóstico para 2 meses adelante
forecast = feature_bagging_model.forecast(X, n_periods=1)


|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
# Convertir la lista a un DataFrame
df_predictions = pd.DataFrame(predictions)
df_predictions.to_csv(DATOS_DIR+'/pred/predicciones.csv', index=False,header=True)
df_predictions.head(10)

,product_id,tn
0,20032,590.333660
1,21153,0.589564
2,21159,0.513512
3,21168,0.488598
4,20286,49.778057
5,20442,31.269903
6,20491,15.661783
7,20548,20.247307
8,20620,15.088071
9,20623,26.421832
